In [1]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm import tqdm
import os

abc 2
def 2
zzz 2
pqr 1


In [83]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [93]:
def calculateIDF(vocab,dataset):  #this method calculates idf and returns idf of all the vocablary it will check each vocab is present in each sentence /n
    count = 0                     #of corpus if yes counter will be increased finally we get in how many senstences a word is present in corpus
    idf_list = dict()
    for key in vocab.keys():
        
        for row in dataset:
            if key in row:
                count = count+1
        IDF = 1 + (math.log((1+len(dataset))/(count+1)))   
        idf_list[key] = IDF
         #idf_list.append(IDF)
        count = 0
    return idf_list

In [94]:
def calculateTFIDF(row,IDF_list_method): #TFIDF is achieved by multiplying idf calculated in previous method with term frequency arugments are each document /n
    final_dict = dict()                  #from coprpus 
    word_freq = dict(Counter(row.split()))
    
    
    for i,j in enumerate(word_freq):
        
        
        if j in IDF_list_method.keys():
           
           final_dict[j]=(word_freq.get(j)/len(word_freq)) * IDF_list_method[j]

        
    
    return final_dict

In [95]:
from tqdm import tqdm # tqdm is a library that helps us to visualize the runtime of for loop. refer this to know more about tqdm
#https://tqdm.github.io/

# it accepts only list of sentances
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    final_IDF = dict()
    vocab = dict()
    IDF_list_method = dict()
    # check if its list type or not
    
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
           
           
            #print(len(final_IDF))
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))  # sorting unique words
        vocab = {j:i for i,j in enumerate(unique_words)} # building vocab dictionary
        IDF_list_method = calculateIDF(vocab,dataset)  #calculating idf for each vocab
        
        
        
        return vocab,IDF_list_method
    else:
        print("you need to pass list of sentance")

In [96]:
def transform(dataset,vocab,IDF_list_method):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            
            # it will return a dict type object where key is the word and values is its frequency, {word:TFIDF}
            #word_freq = dict(Counter(row.split()))
            word_freq=calculateTFIDF(row,IDF_list_method) # return dictionary contains word and TFIDF value

            if len(word_freq) == 0:   # check dictionary is not empty
               continue
            
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.
                         
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    
                    # we are storing the frequency of the word
                    values.append(freq)
                    
                    
                    
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")

In [97]:
vocab,IDF_list_method = fit(corpus)
print(vocab)
print(IDF_list_method)

print(normalize((transform(corpus, vocab, IDF_list_method)), norm='l2') ) #normalizing the sparse matrix



{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}
{'and': 1.916290731874155, 'document': 1.2231435513142097, 'first': 1.5108256237659907, 'is': 1.0, 'one': 1.916290731874155, 'second': 1.916290731874155, 'the': 1.0, 'third': 1.916290731874155, 'this': 1.0}


100%|██████████| 4/4 [00:00<00:00, 14413.42it/s]

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149
  (1, 1)	0.6876235979836937
  (1, 3)	0.2810886740337529
  (1, 5)	0.5386476208856762
  (1, 6)	0.2810886740337529
  (1, 8)	0.2810886740337529
  (2, 0)	0.511848512707169
  (2, 3)	0.267103787642168
  (2, 4)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 7)	0.511848512707169
  (2, 8)	0.267103787642168
  (3, 1)	0.4697913855799205
  (3, 2)	0.580285823684436
  (3, 3)	0.3840852409148149
  (3, 6)	0.3840852409148149
  (3, 8)	0.3840852409148149


In [89]:
import pickle
with open('cleaned_strings', 'rb') as f:
    corpus2 = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus2))

Number of documents in corpus =  746


In [98]:
def Fetch_top50_vocab(vocab,top_50_idf_values):  # aurguments are all the vocab and top 50 idf values will retrn top 50 vocab values with column numbers
    vocab_50 = dict()
    count = -1
    for i,j in enumerate(vocab):
        
        if j in top_50_idf_values.keys():
          count = count + 1
          vocab_50[j] = count
    print(vocab_50)      
    return vocab_50
        
    

In [99]:
from tqdm import tqdm # tqdm is a library that helps us to visualize the runtime of for loop. refer this to know more about tqdm
#https://tqdm.github.io/

# it accepts only list of sentances
def fit_50(dataset):    
    unique_words = set() # at first we will initialize an empty set
    final_IDF = dict()
    vocab = dict()
    IDF_list_method = dict()
    # check if its list type or not
    
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
           
           
            
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        IDF_list_method = calculateIDF(vocab,dataset)
        
        sorted_dict = {k: v for k,v in sorted(IDF_list_method.items(), key=lambda v: v[1], reverse=True)} #sorting dictionary based on values in descending order
        
        
        first_50 = {k: sorted_dict[k] for k in list(sorted_dict)[:50]} # retriving top 50 idf values 
        
        vocab_50 = Fetch_top50_vocab(vocab,first_50)  # retriving top 50 vocabvalues with column numbers
        
        
        
        return vocab_50,first_50  #returning top 50 idf and vocab values
    else:
        print("you need to pass list of sentance")

In [100]:
vocab_50,first_50 = fit_50(corpus2)

print(normalize((transform(corpus2, vocab_50, first_50)), norm='l2') ) # normalizing sparse transform output


{'aailiyah': 0, 'abandoned': 1, 'abroad': 2, 'abstruse': 3, 'academy': 4, 'accents': 5, 'accessible': 6, 'acclaimed': 7, 'accolades': 8, 'accurately': 9, 'achille': 10, 'ackerman': 11, 'adams': 12, 'added': 13, 'admins': 14, 'admiration': 15, 'admitted': 16, 'adrift': 17, 'adventure': 18, 'aesthetically': 19, 'affected': 20, 'affleck': 21, 'afternoon': 22, 'agreed': 23, 'aimless': 24, 'aired': 25, 'akasha': 26, 'alert': 27, 'alike': 28, 'allison': 29, 'allowing': 30, 'alongside': 31, 'amateurish': 32, 'amazed': 33, 'amazingly': 34, 'amusing': 35, 'amust': 36, 'anatomist': 37, 'angela': 38, 'angelina': 39, 'angry': 40, 'anguish': 41, 'angus': 42, 'animals': 43, 'animated': 44, 'anita': 45, 'anniversary': 46, 'anthony': 47, 'antithesis': 48, 'anyway': 49}


100%|██████████| 746/746 [00:00<00:00, 101103.49it/s]

  (0, 24)	1.0
  (19, 43)	1.0
  (68, 21)	1.0
  (72, 23)	1.0
  (74, 25)	1.0
  (89, 47)	1.0
  (135, 3)	0.3779644730092272
  (135, 9)	0.3779644730092272
  (135, 15)	0.3779644730092272
  (135, 17)	0.3779644730092272
  (135, 29)	0.3779644730092272
  (135, 32)	0.3779644730092272
  (135, 40)	0.3779644730092272
  (176, 39)	1.0
  (192, 18)	1.0
  (193, 20)	1.0
  (216, 2)	1.0
  (225, 16)	1.0
  (227, 14)	1.0
  (241, 35)	1.0
  (270, 1)	1.0
  (290, 22)	1.0
  (341, 34)	1.0
  (344, 33)	1.0
  (348, 8)	1.0
  (409, 5)	1.0
  (430, 31)	1.0
  (457, 36)	1.0
  (461, 4)	0.7071067811865475
  (461, 44)	0.7071067811865475
  (465, 30)	1.0
  (475, 28)	1.0
  (493, 6)	1.0
  (500, 38)	1.0
  (544, 41)	1.0
  (548, 0)	0.7071067811865475
  (548, 26)	0.7071067811865475
  (608, 12)	1.0
  (612, 10)	1.0
  (620, 37)	0.7071067811865476
  (620, 42)	0.7071067811865476
  (632, 7)	1.0
  (644, 11)	0.5773502691896257
  (644, 45)	0.5773502691896257
  (644, 46)	0.5773502691896257
  (667, 19)	1.0
  (691, 27)	1.0
  (699, 48)	1.0
  (722, 1

**reference from the refernce_notebook provided as a part of assignment**